<a href="https://colab.research.google.com/github/Acheon-stst/Acheon-stst/blob/main/AndreCehon__DNSC4280_Final_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Data source:

[bitcoin-tweets-spam-emotion-sentiment](https://huggingface.co/datasets/sandiumenge/bitcoin-tweets-spam-emotion-sentiment)

- Renamed data files:
  - train-00000-of-00001.parquet    ->    TrainBitcoin.parquet
  - validation-00000-of-00001.parquet    ->     ValidateBitcoin.parquet
  - test-00000-of-00001.parquet     ->    TestBitcoin.parquet

# Data Preprocessing

## Reading in Data

In [17]:
import pandas as pd

In [ ]:
train = pd.DataFrame(pd.read_parquet("TrainBitcoin.parquet"))
val = pd.DataFrame(pd.read_parquet("ValidateBitcoin.parquet"))
test = pd.DataFrame(pd.read_parquet("TestBitcoin.parquet"))

In [ ]:
# Analyzing the completeness of the datasets:

print(train.info())
print("")
print(val.info())
print("")
print(test.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 76452 entries, 0 to 76451
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   text       76452 non-null  object 
 1   spam       76452 non-null  object 
 2   emotion    76452 non-null  object 
 3   sentiment  76452 non-null  float64
dtypes: float64(1), object(3)
memory usage: 2.3+ MB
None

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16383 entries, 0 to 16382
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   text       16383 non-null  object 
 1   spam       16383 non-null  object 
 2   emotion    16383 non-null  object 
 3   sentiment  16383 non-null  float64
dtypes: float64(1), object(3)
memory usage: 512.1+ KB
None

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16383 entries, 0 to 16382
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  ----- 

## Text Preprocessing

In [ ]:
!pip install gensim

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.9/27.9 MB 46.6 MB/s eta 0:00:00


In [ ]:
import gensim.downloader as api
import nltk
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('punkt_tab')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


True

In [ ]:
import string

In [ ]:
# Converting all text to lowercase.

train["modified"] = train["text"].str.lower()
val["modified"] = val["text"].str.lower()
test["modified"] = test["text"].str.lower()

In [ ]:
# Source - https://stackoverflow.com/a
# Posted by nickyfot
# Retrieved 2025-11-09, License - CC BY-SA 4.0

def remove_tag(text): # Removing tags from Tweets (anything that begins with "@"" to tag another user)
    text = text.split()
    text_new = [x for x in text if not x.startswith("@")]
    return " ".join(text_new)

def remove_link(text): # Removing hyperlinks from Tweets (anything that begins with "http")
  text = text.split()
  text_new = [x for x in text if not x.startswith("http")]
  return " ".join(text_new)

train["modified"] = train["modified"].apply(remove_tag)
train["modified"] = train["modified"].apply(remove_link)
val["modified"] = val["modified"].apply(remove_tag)
val["modified"] = val["modified"].apply(remove_link)
test["modified"] = test["modified"].apply(remove_tag)
test["modified"] = test["modified"].apply(remove_link)

In [ ]:
!pip install emoji

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 608.4/608.4 kB 11.0 MB/s eta 0:00:00


In [ ]:
import emoji

In [ ]:
# Removing emojis from the text

def remove_emojis(text):
    return emoji.replace_emoji(text, replace='')

train["modified"] = train["modified"].apply(remove_emojis)
train["modified"] = train["modified"].apply(remove_emojis)
val["modified"] = val["modified"].apply(remove_emojis)
val["modified"] = val["modified"].apply(remove_emojis)
test["modified"] = test["modified"].apply(remove_emojis)
test["modified"] = test["modified"].apply(remove_emojis)

In [ ]:
# Removing punctuation from all text.
# Guide for removing punctuation: https://stackoverflow.com/questions/39782418/remove-punctuations-in-pandas
# David C's comment.

train["modified"] = train["modified"].str.replace("[{}]".format(string.punctuation), "", regex= True)
val["modified"] = val["modified"].str.replace("[{}]".format(string.punctuation), "", regex= True)
test["modified"] = test["modified"].str.replace("[{}]".format(string.punctuation), "", regex= True)

In [ ]:
# Tokenizing the data.
from nltk.tokenize import word_tokenize

train["modified"] = train["modified"].apply(word_tokenize)
val["modified"] = val["modified"].apply(word_tokenize)
test["modified"] = test["modified"].apply(word_tokenize)

In [ ]:
# Removing stop words.

from nltk.corpus import stopwords

stops = stopwords.words("english")
stops = [i.lower() for i in stops]

train["modified"] = train["modified"].apply(lambda x: [word for word in x if word not in stops])
val["modified"] = val["modified"].apply(lambda x: [word for word in x if word not in stops])
test["modified"] = test["modified"].apply(lambda x: [word for word in x if word not in stops])

In [ ]:
from nltk.stem import WordNetLemmatizer
nltk.download("wordnet")
nltk.download("omw-1.4")

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


True

In [ ]:
# Lemmatizing words to get full stems of words using different conjugations.

wnlemm = WordNetLemmatizer()

train["modified"] = train["modified"].apply(lambda x: [wnlemm.lemmatize(word.lower(), pos = "v") for word in x])
val["modified"] = val["modified"].apply(lambda x: [wnlemm.lemmatize(word.lower(), pos = "v") for word in x])
test["modified"] = test["modified"].apply(lambda x: [wnlemm.lemmatize(word.lower(), pos = "v") for word in x])

#RNN Model

The following RNN model format is derived from the source linked below:

https://www.analyticsvidhya.com/blog/2022/09/understanding-word-embeddings-and-building-your-first-rnn-model/#h-building-your-first-rnn-model-for-text-classification-tasks

## Preprocessing

In [ ]:
from sklearn.preprocessing import LabelEncoder

In [ ]:
encoder = LabelEncoder() # Used to encode the y categories as numerical values to be used in the model

# Training sets and labels
Xtrain = train["modified"]
ytrain = encoder.fit_transform(train["spam"])


# Validation sets and labels
Xval = val["modified"]
yval = encoder.fit_transform(val["spam"])


# Test sets and labels
Xtest = test["modified"]
ytest = encoder.fit_transform(test["spam"])

In [ ]:
from tensorflow.keras.preprocessing.text import Tokenizer
from nltk.tokenize import word_tokenize, sent_tokenize

In [ ]:
# Tokenizing the text to convert the words to integer values

tokenizer = Tokenizer() # Initializing the tokenizer

tokenizer.fit_on_texts(Xtrain) # Fitting the tokenizer to the vocabulary in the training data

In [ ]:
from tensorflow.keras.utils import pad_sequences

In [ ]:
# Converting training token integers to lists of words
train_sequence = tokenizer.texts_to_sequences(Xtrain)

# Padding the training data
train_pad_sequence = pad_sequences(train_sequence)

In [ ]:
# Converting validation token integers to lists of words
val_sequence = tokenizer.texts_to_sequences(Xval)

# Padding the validation data
val_pad_sequence = pad_sequences(val_sequence)

In [ ]:
# Converting test token integers to lists of words
test_sequence = tokenizer.texts_to_sequences(Xtest)

# Padding the test data
test_pad_sequence = pad_sequences(test_sequence)

In [ ]:
word_index = tokenizer.word_index # Corresponding index for each word token
max_words = 30000 # Number of words to be considered in the model
total_words = len(word_index) # Total number of words in the data

In [ ]:
from tensorflow.keras.utils import to_categorical

In [ ]:
# Converting output data to categorical values

# Training y values
ytrain = to_categorical(ytrain, num_classes = 3)

# Validation y values
yval = to_categorical(yval, num_classes = 3)

# Test y values
ytest = to_categorical(ytest, num_classes = 3)


In [ ]:
longest_sequence = max(train_pad_sequence, key=len)
len(longest_sequence)

1079

In [ ]:
shortest_sequence = min(train_pad_sequence, key=len)
len(shortest_sequence)

1079

In [ ]:
max_len = 1079 # Based on the padded lengths of the sequences

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import SimpleRNN, Dense
from tensorflow.keras.layers import Embedding, Bidirectional, LSTM
from tensorflow.keras.layers import Dropout

## Tuned RNN Model

Utilizing the Keras Tuner to find an optimal model configuration to predict the "spam" column of the data:
https://www.tensorflow.org/tutorials/keras/keras_tuner

In [ ]:
pip install keras-tuner

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.4/129.4 kB 3.7 MB/s eta 0:00:00


In [ ]:
import keras_tuner as kt
from tensorflow import keras
from tensorflow.keras import layers

In [ ]:
def build_rnn(hp): # Developing the hyperparameters for the keras_tuner to
  tuned_rnn_spam = keras.Sequential() # Limiting the input sequence length to 500 tokens; exploring 32-128 units for the embedding layer
  tuned_rnn_spam.add(Embedding((total_words+1), output_dim=hp.Int("embedding_dim", min_value=32, max_value=128, step=32), input_length=500)) # Input embedding layer

  for i in range (hp.Int("num_hidden_layers", 1, 3)): # Optimizing the hidden layers to add; one to three hidden layers with neurons ranging from 32-128
    tuned_rnn_spam.add(layers.Bidirectional(layers.SimpleRNN(units=hp.Int(f"units_{i}", min_value=32, max_value=128, step=32),
                                                        return_sequences=True if i < hp.Int("num_hidden_layers", 1, 3) - 1 else False,
                                                        activation="tanh")))
    tuned_rnn_spam.add(layers.Dropout(hp.Float(f"dropout_{i}", min_value=0.0, max_value=0.4, step=0.1)))

  for i in range(hp.Int("num_dense_layers", 1, 2)): # Optimizing the number of dense layers to add
    tuned_rnn_spam.add(layers.Dense(units=hp.Int(f"dense_units_{i}", min_value = 32, max_value=128, step=32))) # Optimizing the number of dense units to add

    tuned_rnn_spam.add((layers.Dropout(hp.Float(f"dropout_{i}", min_value=0.0, max_value=0.4, step=0.1))))

  tuned_rnn_spam.add(Dense(3, activation="softmax")) # Using the softmax activation function and optimizing its learning rate
  learning_rate = hp.Float("learning_rate", min_value=1e-4, max_value=1e-2, sampling="log")

  # Compiling the spam detection RNN model
  tuned_rnn_spam.compile(optimizer="rmsprop",loss="categorical_crossentropy",metrics=["accuracy"]) # Going based on model accuracy to determine the best model

  return tuned_rnn_spam

In [ ]:
tuner_spam = kt.Hyperband(build_rnn, # Using a hyperband tuner to optimize the model hyperparameters
                        objective="val_accuracy", # Maximize validation accuracy
                        max_epochs=5, # Using only 5 epochs due to time constraints
                        hyperband_iterations=1,
                        executions_per_trial=1,
                        factor=5, # Using a factor of 5 to quickly cut out ineffective models due to computation resource availability and time constraints
                        overwrite=True)

/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/embedding.py:97: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


In [ ]:
# Searching for the best hyperparameter configuration to use with this data

tuner_spam.search(train_pad_sequence, ytrain, epochs=5, batch_size = 256, validation_data=(val_pad_sequence, yval))

Trial 8 Complete [00h 30m 14s]
val_accuracy: 0.8030275106430054

Best val_accuracy So Far: 0.8030275106430054
Total elapsed time: 02h 30m 57s


In [ ]:
best_model = tuner_spam.get_best_models(num_models=1)[0] # Obtaining the best model's configuration

In [ ]:
best_hps = tuner_spam.get_best_hyperparameters(num_trials=1)[0] # Saving the best model's hyperparameters
best_model = tuner_spam.get_best_models(num_models=1)[0] # Saving the best model (according to validation performance)

In [ ]:
best_hps.values # Values for hyperparameters used in the best-performing model

{'embedding_dim': 128,
 'num_hidden_layers': 1,
 'units_0': 96,
 'dropout_0': 0.0,
 'num_dense_layers': 1,
 'dense_units_0': 96,
 'learning_rate': 0.0018492573544654537,
 'units_1': 32,
 'dropout_1': 0.0,
 'dense_units_1': 32,
 'units_2': 128,
 'dropout_2': 0.0,
 'tuner/epochs': 5,
 'tuner/initial_epoch': 0,
 'tuner/bracket': 0,
 'tuner/round': 0}

In [ ]:
# Best model's accuracy and loss values on the test data

test_loss, test_accuracy = best_model.evaluate(test_pad_sequence, ytest)
print(f"Best model test accuracy: {test_accuracy}")
print(f"Best model test loss: {test_loss}")

512/512 ━━━━━━━━━━━━━━━━━━━━ 4s 5ms/step - accuracy: 0.8029 - loss: 0.5079
Best model test accuracy: 0.8005859851837158
Best model test loss: 0.5166557431221008


In [ ]:
import numpy as np
from sklearn.metrics import classification_report

In [ ]:
val_y_pred_probs = best_model.predict(val_pad_sequence) # Predicted probabilities for the validation set's labels
val_y_pred = np.argmax(val_y_pred_probs, axis=1) # Converting the probabilities to predicted labels
val_y_true = np.argmax(yval, axis=1) # True validation labels

512/512 ━━━━━━━━━━━━━━━━━━━━ 10s 18ms/step


In [ ]:
# Classification report for the validation data

target_names = ['human', 'spam', 'bot']
print(classification_report(val_y_true, val_y_pred, target_names=target_names))

              precision    recall  f1-score   support

       human       0.72      0.69      0.71      3718
        spam       0.83      0.87      0.85      8762
         bot       0.81      0.75      0.78      3903

    accuracy                           0.80     16383
   macro avg       0.79      0.77      0.78     16383
weighted avg       0.80      0.80      0.80     16383



In [ ]:
y_pred_probs = best_model.predict(test_pad_sequence) # Predicted probabilities for the test set's labels
y_pred = np.argmax(y_pred_probs, axis=1) # Converting the probabilities to predicted labels
y_true = np.argmax(ytest, axis=1) # True test labels

512/512 ━━━━━━━━━━━━━━━━━━━━ 3s 6ms/step


In [ ]:
# Classification report for the test data

target_names = ['human', 'spam', 'bot']
print(classification_report(y_true, y_pred, target_names=target_names))

              precision    recall  f1-score   support

       human       0.78      0.59      0.67      3716
        spam       0.80      0.91      0.85      8695
         bot       0.80      0.77      0.78      3972

    accuracy                           0.80     16383
   macro avg       0.80      0.75      0.77     16383
weighted avg       0.80      0.80      0.80     16383



In [ ]:
from sklearn.metrics import accuracy_score

val_accuracy = accuracy_score(val_y_true, val_y_pred) # Overall accuracy score on the validation data
test_accuracy = accuracy_score(y_true, y_pred) # Overall accuracy score on the test data

print(f"Validation accuracy: {val_accuracy}")
print(f"Test accuracy: {test_accuracy}")

Validation accuracy: 0.8030275285356773
Test accuracy: 0.8005859732649698


#Transformer

The following transformer model format is derived from the source linked below:  

https://achimoraites.medium.com/fine-tuning-roberta-for-topic-classification-with-hugging-face-transformers-and-datasets-library-c6f8432d0820

## Pre-Processing

In [ ]:
import pandas as pd

# Reading in the data files
train = pd.DataFrame(pd.read_parquet("TrainBitcoin.parquet"))
val = pd.DataFrame(pd.read_parquet("ValidateBitcoin.parquet"))
test = pd.DataFrame(pd.read_parquet("TestBitcoin.parquet"))

In [ ]:
# Dropping the unused columns

train_t = train.drop(columns=["emotion", "sentiment"])
val_t = val.drop(columns=["emotion", "sentiment"])
test_t = test.drop(columns=["emotion", "sentiment"])

In [ ]:
# Renaming the "spam" column to "label" for easier use with the pretrained transformer

train_t = train_t.rename(columns={'spam': 'label'})
val_t = val_t.rename(columns={'spam': 'label'})
test_t = test_t.rename(columns={'spam': 'label'})

In [ ]:
import torch
from datasets import load_dataset
from transformers import (
    RobertaTokenizerFast,
    RobertaForSequenceClassification,
    TrainingArguments,
    Trainer,
    AutoConfig,
)

In [ ]:
from datasets import Dataset

In [ ]:
# Converting the pandas dataframes to Hugging Face datasets so they work with the Hugging Face transformers

train_t = Dataset.from_pandas(train_t)
val_t = Dataset.from_pandas(val_t)
test_t = Dataset.from_pandas(test_t)

In [ ]:
model_id = "roberta-base" # Pretrained model name that will be used

In [ ]:
tokenizer = RobertaTokenizerFast.from_pretrained(model_id) # Initializing the tokenizer to be used on the datasets

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/481 [00:00<?, ?B/s]

In [ ]:
# Tokenizer function to tokenize the the "text" column of the datasets

def tokenize(batch):
    return tokenizer(batch["text"], padding=True, truncation=True, max_length=500)

# Tokenizing the text in each of the datasets

train_t = train_t.map(tokenize, batched=True, batch_size=len(train_t))
val_t = val_t.map(tokenize, batched=True, batch_size=len(val_t))
test_t = test_t.map(tokenize, batched=True, batch_size=len(test_t))

Map:   0%|          | 0/76452 [00:00<?, ? examples/s]

Map:   0%|          | 0/16383 [00:00<?, ? examples/s]

Map:   0%|          | 0/16383 [00:00<?, ? examples/s]

In [ ]:
# Set dataset format to be compatible with PyTorch

train_t.set_format("torch", columns=["input_ids", "attention_mask", "label"])
val_t.set_format("torch", columns=["input_ids", "attention_mask", "label"])
test_t.set_format("torch", columns=["input_ids", "attention_mask", "label"])

In [ ]:
# Initializing the number of class labels and their names
num_labels = 3
class_names = ["human", "spam", "bot"]

# Create an id2label mapping
id2label = {i: label for i, label in enumerate(class_names)}

# Updating the model's configuration with the id2label mapping
config = AutoConfig.from_pretrained(model_id)
config.update({"id2label": id2label})

In [ ]:
# Encoding each of the class names to be numbers so they are compatible with the transformer model

train_t = train_t.class_encode_column("label")
val_t = val_t.class_encode_column("label")
test_t = test_t.class_encode_column("label")

Casting to class labels:   0%|          | 0/76452 [00:00<?, ? examples/s]

Casting to class labels:   0%|          | 0/16383 [00:00<?, ? examples/s]

Casting to class labels:   0%|          | 0/16383 [00:00<?, ? examples/s]

## Initializing Model

In [ ]:
# Initializing the RoBERTa model

model = RobertaForSequenceClassification.from_pretrained(model_id, config=config)


model.safetensors:   0%|          | 0.00/499M [00:00<?, ?B/s]

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
# Training argument specifications

training_args = TrainingArguments(
    do_eval=True,
    num_train_epochs=3,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    eval_strategy="epoch",
    logging_strategy="steps",
    logging_steps=10,
    learning_rate=5e-5,
    weight_decay=0.01,
    warmup_steps=500,
    save_strategy="epoch",
    load_best_model_at_end=True,
    save_total_limit=2,
    report_to="none",
    dataloader_pin_memory=False
)


In [ ]:
# Due to computational resource availability and time constraints, the datasets
# will be dropped to 7000 random values each. This action will speed up the training
# process so we can at least see how the transformer works with our data.

train_dataset = train_t.shuffle(seed=42).select(range(7000))
val_dataset = val_t.shuffle(seed=42).select(range(7000))

In [ ]:
# Trainer setup

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset
)

In [ ]:
# Fine-tuning the model through training

trainer.train()

Epoch,Training Loss,Validation Loss
1,0.346100,0.691111
2,0.362200,0.535625
3,0.298500,0.624597


TrainOutput(global_step=2625, training_loss=0.46839612462407065, metrics={'train_runtime': 44116.0191, 'train_samples_per_second': 0.476, 'train_steps_per_second': 0.06, 'total_flos': 5395880637000000.0, 'train_loss': 0.46839612462407065, 'epoch': 3.0})

## Performance on Validation and Test Data

In [ ]:
import numpy as np
from sklearn.metrics import classification_report

In [ ]:
val_predictions = trainer.predict(val_dataset) # Predicting the validation dataset labels

In [ ]:
val_y_true = val_predictions.label_ids # Validation true labels
val_y_pred_logits = val_predictions.predictions # Validation predictions through transformer
val_y_pred = np.argmax(val_y_pred_logits, axis=1) # Converting logit predictions to class labels

In [ ]:
# Validation classification matrix

print(classification_report(val_y_true, val_y_pred, target_names=class_names))

              precision    recall  f1-score   support

       human       0.71      0.87      0.78      1614
        spam       0.91      0.86      0.88      3767
         bot       0.85      0.76      0.80      1619

    accuracy                           0.84      7000
   macro avg       0.82      0.83      0.82      7000
weighted avg       0.85      0.84      0.84      7000



In [ ]:
test_dataset = test_t.shuffle(seed=42).select(range(7000)) # Creating a smaller test set for predictions

In [ ]:
predictions = trainer.predict(test_dataset) # Predicting the test dataset labels

In [ ]:
y_true = predictions.label_ids # Test true labels
y_pred_logits = predictions.predictions # Test predictions through the transformer
y_pred = np.argmax(y_pred_logits, axis=1) # Converting logit predictions to class lables

In [ ]:
# Test classification matrix

print(classification_report(y_true, y_pred, target_names=class_names))


              precision    recall  f1-score   support

       human       0.71      0.86      0.78      1595
        spam       0.90      0.87      0.88      3709
         bot       0.86      0.75      0.80      1696

    accuracy                           0.84      7000
   macro avg       0.82      0.83      0.82      7000
weighted avg       0.85      0.84      0.84      7000



In [ ]:
from sklearn.metrics import accuracy_score

val_accuracy = accuracy_score(val_y_true, val_y_pred) # Overall accuracy score on the validation data
test_accuracy = accuracy_score(y_true, y_pred) # Overall accuracy score on the test data

print(f"Validation accuracy: {val_accuracy}")
print(f"Test accuracy: {test_accuracy}")

Validation accuracy: 0.8411428571428572
Test accuracy: 0.8392857142857143


# Logistic Regression

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix

# Convert tokenized tweets back to strings
Xtrain_text = train['modified'].apply(lambda x: ' '.join(x))
Xval_text = val['modified'].apply(lambda x: ' '.join(x))
Xtest_text = test['modified'].apply(lambda x: ' '.join(x))

# Vectorize text using TF-IDF
tfidf = TfidfVectorizer(max_features=10000, ngram_range=(1,2))
Xtrain_tfidf = tfidf.fit_transform(Xtrain_text)
Xval_tfidf = tfidf.transform(Xval_text)
Xtest_tfidf = tfidf.transform(Xtest_text)




In [ ]:
from sklearn.preprocessing import LabelEncoder

# Encode labels as integers (spam/bot/human)
encoder = LabelEncoder()
ytrain_int = encoder.fit_transform(train["spam"])
yval_int   = encoder.transform(val["spam"])
ytest_int  = encoder.transform(test["spam"])



In [ ]:
# Logistic Regression model
lr_model = LogisticRegression(max_iter=500)
lr_model.fit(Xtrain_tfidf, ytrain_int)

# Validation predictions
lr_val_pred = lr_model.predict(Xval_tfidf)
print("Validation Classification Report:")
print(classification_report(yval_int, lr_val_pred))

# Test predictions
lr_test_pred = lr_model.predict(Xtest_tfidf)
print("Test Classification Report:")
print(classification_report(ytest_int, lr_test_pred))


Validation Classification Report:
              precision    recall  f1-score   support

           0       0.74      0.70      0.72      3718
           1       0.84      0.88      0.86      8762
           2       0.82      0.77      0.79      3903

    accuracy                           0.81     16383
   macro avg       0.80      0.78      0.79     16383
weighted avg       0.81      0.81      0.81     16383

Test Classification Report:
              precision    recall  f1-score   support

           0       0.74      0.70      0.72      3716
           1       0.83      0.89      0.86      8695
           2       0.84      0.76      0.80      3972

    accuracy                           0.81     16383
   macro avg       0.80      0.78      0.79     16383
weighted avg       0.81      0.81      0.81     16383



# Linear SVM


In [ ]:
from sklearn.svm import LinearSVC

svm_model = LinearSVC()
svm_model.fit(Xtrain_tfidf, ytrain_int)

svm_val_pred  = svm_model.predict(Xval_tfidf)
svm_test_pred = svm_model.predict(Xtest_tfidf)

# Validation predictions
print("SVM_Validation Classification Report:")
print(classification_report(yval_int, svm_val_pred))

# Test predictions
print("SVM_Test Classification Report:")
print(classification_report(ytest_int, svm_test_pred))





SVM_Validation Classification Report:
              precision    recall  f1-score   support

           0       0.73      0.70      0.72      3718
           1       0.84      0.87      0.85      8762
           2       0.81      0.76      0.79      3903

    accuracy                           0.81     16383
   macro avg       0.79      0.78      0.79     16383
weighted avg       0.81      0.81      0.81     16383

SVM_Test Classification Report:
              precision    recall  f1-score   support

           0       0.72      0.70      0.71      3716
           1       0.83      0.87      0.85      8695
           2       0.82      0.76      0.79      3972

    accuracy                           0.81     16383
   macro avg       0.79      0.78      0.78     16383
weighted avg       0.80      0.81      0.80     16383



# GloVe CNN


In [ ]:
# Converting token lists back to strings
train_text = train["modified"].apply(lambda x: " ".join(x))
val_text   = val["modified"].apply(lambda x: " ".join(x))
test_text  = test["modified"].apply(lambda x: " ".join(x))


In [ ]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.utils import pad_sequences

max_words = 10000
max_len = 100

tokenizer = Tokenizer(num_words=max_words, oov_token="<OOV>")
tokenizer.fit_on_texts(Xtrain_text)

Xtrain_seq = tokenizer.texts_to_sequences(Xtrain_text)
Xval_seq   = tokenizer.texts_to_sequences(Xval_text)
Xtest_seq  = tokenizer.texts_to_sequences(Xtest_text)

Xtrain_pad = pad_sequences(Xtrain_seq, maxlen=max_len)
Xval_pad   = pad_sequences(Xval_seq, maxlen=max_len)
Xtest_pad  = pad_sequences(Xtest_seq, maxlen=max_len)

word_index = tokenizer.word_index

In [ ]:
# Load GloVe Twitter Embeddings + Build Embedding Matrix

# Load pretrained GloVe Twitter vectors (100 dimensions)
glove = api.load("glove-twitter-100")
embedding_dim = 100

embedding_matrix = np.zeros((max_words, embedding_dim))

for word, i in word_index.items():
    if i < max_words and word in glove:
        embedding_matrix[i] = glove[word]



In [ ]:
# Train the GloVe CNN
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Conv1D, GlobalMaxPooling1D, Dense, Dropout

model = Sequential([
    Embedding(
        input_dim=max_words,
        output_dim=embedding_dim,
        input_length=max_len,
        weights=[embedding_matrix],
        trainable=False
    ),
    Conv1D(128, 5, activation='relu'),
    Conv1D(128, 5, activation='relu'),
    GlobalMaxPooling1D(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(3, activation='softmax') # 3 classes: spam / bot / human
])

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

ytrain_onehot = to_categorical(ytrain_int, num_classes=3)
yval_onehot   = to_categorical(yval_int, num_classes=3)

history = model.fit(
    Xtrain_pad, ytrain_onehot,
    validation_data=(Xval_pad, yval_onehot),
    epochs=8,
    batch_size=128
)

NameError: name 'embedding_dim' is not defined

In [ ]:
# Validation
CNN_val_pred = model.predict(Xval_pad).argmax(axis=1)
print("Validation Classification Report:")
print(classification_report(yval_int, CNN_val_pred))

# Test
CNN_test_pred = model.predict(Xtest_pad).argmax(axis=1)
print("Test Classification Report:")
print(classification_report(ytest_int, CNN_test_pred))